In [19]:
import numpy as np
import pandas as pd
import os
import requests
from difflib import SequenceMatcher
from tqdm import tqdm_notebook
from time import sleep
import ast

In [20]:
def loadCoinData():
    dataList = []
    tokenFileList = os.listdir('./tokenData')
    
    for token in tqdm_notebook(tokenFileList):
        tokenData = pd.read_csv('./tokenData/'+token, skiprows=1).set_index('id')
        dataList.append(tokenData)
        
    return pd.concat(dataList, axis=1).T
    

In [21]:
def hasCategory(x):
    if len(ast.literal_eval(x)) > 0:
        return 1
    else:
        return 0

In [22]:
def hasSpecificCategory(x, category):
    if category in x:
        return 1
    else:
        return 0

In [23]:
def hasBaseCategory(x, catArr):
    
#     print(set(catArr), set(ast.literal_eval(x)))
#     print(set(catArr).intersection(ast.literal_eval(x)))
#     print(any(x in catArr for x in ast.literal_eval(x)))

    if len(set(catArr).intersection(ast.literal_eval(x))) > 0:
        return 1
    else:
        return 0

In [24]:
def getTokensWithCategory(coinDF):
    coinDF['has_category'] = coinDF['categories'].map(lambda x: hasCategory(x))
    return coinDF[coinDF['has_category'] == 1].drop(columns=['has_category'])


In [25]:
def getAllCategories(coinDF):
    cat_list = [ast.literal_eval(x) for x in list(coinDF['categories'].values)]
    all_categories = set([item for sublist in cat_list for item in sublist])
    return all_categories

In [26]:
def getCategoryCounts(coinDF, all_categories):
    categoryTotals = []
    for cat in all_categories:
        categoryCount = coinDF['categories'].map(lambda x: hasSpecificCategory(x, cat))
        categoryTotals.append(categoryCount.sum())
        
    return pd.Series(categoryTotals, index=all_categories).sort_values(ascending=False)

In [27]:
def getCoinsWithCategory(coinDF, category):
    coinDF['has_given_category'] = coinDF['categories'].map(lambda x: hasSpecificCategory(x, category))
    return coinDF[coinDF['has_given_category'] > 0].drop(columns=['has_given_category'])


In [28]:
def getCoinsInBaseCategory(coinDF, catArr):
    coinDF['has_given_category'] = coinDF['categories'].map(lambda x: hasBaseCategory(x, catArr))
    return coinDF[coinDF['has_given_category'] > 0].drop(columns=['has_given_category'])


In [29]:
def mapTrustScore(score):
    if score == 'green':
        return 1
    if score == 'yellow':
        return 0
    if score == 'red':
        return -1

In [30]:
def getNumericalQualitative(CC, useColumns):
    DF = CC[useColumns].copy()
    DF['trust_score'] = DF['trust_score'].map(lambda x: mapTrustScore(x))
    return DF

In [31]:
def describeCategory(catCoinDF, category, useColumns):
    CC = getCoinsInBaseCategory(catCoinDF, baseCategories[category])
    NQ = getNumericalQualitative(CC, useColumns)
    print('Number of tokens:', CC.shape[0])
    return NQ.astype('float').fillna(0).describe()

### run scripts

In [32]:
coinDF = loadCoinData()

<ipython-input-20-84562f8f934e>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for token in tqdm_notebook(tokenFileList):


In [33]:
catCoinDF = getTokensWithCategory(coinDF)

In [34]:
coinDF.shape

(5713, 46)

In [35]:
catCoinDF.shape

(2269, 45)

In [36]:
all_categories = getAllCategories(catCoinDF)

In [37]:
categoryTotals = getCategoryCounts(coinDF, all_categories)

In [38]:
pd.set_option('display.max_rows', 100)

In [48]:
categoryTotals.shape

(107,)

In [39]:
categoryTotals[categoryTotals > 10]

Decentralized Finance (DeFi)          308
Polygon Ecosystem                     209
Non-Fungible Tokens (NFT)             199
Finance / Banking                     165
Cryptocurrency                        152
Yield Farming                         140
Exchange-based Tokens                 137
Binance Smart Chain Ecosystem         132
Decentralized Exchange Token (DEX)    102
Governance                            101
Number                                101
RealT Tokens                           98
Business Services                      97
Asset-backed Tokens                    88
Business Platform                      87
Entertainment                          72
Infrastructure                         59
Polkadot Ecosystem                     58
Protocol                               54
Meme Tokens                            51
xDAI Ecosystem                         45
Stablecoins                            42
Centralized Exchange Token (CEX)       39
Smart Contract Platform           

In [40]:
baseCategories = {
    'entertainment': ['Non-Fungible Tokens (NFT)', 'Entertainment'],
    'social': ['Gaming', 'Communication', 'Fan Token', 'Media', 'Sports', 'Tourism', 'Charity', 'Collectible', 'Social Money'],
    'defi': ['Launchpad', 'Lending/Borrowing', 'Automated Market Maker (AMM)', 'Seigniorage'],
    'business': ['Business Platform', 'Energy', 'Big Data', 'Software', 'Insurance', 'Retail', 'Real Estate'],
    'blockchain': ['Smart Contract Platform','Protocol','Oracle'],
    'meme': ['Meme Tokens']
}

In [41]:
useColumns = ['sentiment_votes_up_percentage', 'sentiment_votes_down_percentage','market_cap_rank',\
              'coingecko_rank', 'coingecko_score', 'developer_score', 'community_score', 'liquidity_score', \
              'public_interest_score', 'facebook_likes', 'twitter_followers', \
              'reddit_subscribers','reddit_accounts_active_48h', 'telegram_channel_user_count', \
              'mean', 'std', 'kurt', 'trust_score']

#### describe category

In [42]:
for k, v in baseCategories.items():
    CC = getCoinsInBaseCategory(catCoinDF, v)
    CC.to_csv('./categoryLists/'+k+'_token_list.csv')


### entertainment

In [43]:
describeCategory(catCoinDF, 'entertainment', useColumns)

Number of tokens: 257


id,sentiment_votes_up_percentage,sentiment_votes_down_percentage,market_cap_rank,coingecko_rank,coingecko_score,developer_score,community_score,liquidity_score,public_interest_score,facebook_likes,twitter_followers,reddit_subscribers,reddit_accounts_active_48h,telegram_channel_user_count,mean,std,kurt,trust_score
count,257.000000,257.000000,257.000000,257.000000,257.000000,257.000000,257.000000,257.000000,257.000000,257.0,257.000000,257.000000,257.000000,257.000000,257.000000,257.000000,257.000000,257.000000
mean,52.576420,13.960545,1054.136187,2762.715953,13.906039,3.499537,11.721350,10.413381,0.014724,0.0,26369.058366,793.669261,15.420233,7864.287938,6.334631,225.847874,89.130659,0.431907
std,42.313589,22.207831,880.590649,1912.846783,8.799194,11.194509,9.819131,14.250599,0.049794,0.0,60823.210031,4457.718044,166.346807,15875.829084,99.927069,3586.303736,191.686342,0.634485
min,0.000000,0.000000,0.000000,58.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.035895,0.021854,0.130123,-1.000000
25%,0.000000,0.000000,111.000000,1400.000000,9.330000,0.000000,7.331000,1.000000,0.001000,0.0,1060.000000,0.000000,0.000000,184.000000,0.000833,0.105324,6.058439,0.000000
50%,66.670000,0.000000,1023.000000,2421.000000,12.922000,0.000000,8.979000,1.000000,0.003000,0.0,8987.000000,0.000000,0.000000,1947.000000,0.009630,0.155115,17.877359,1.000000
75%,90.910000,20.000000,1843.000000,3268.000000,18.436000,0.000000,10.861000,17.547000,0.008000,0.0,24104.000000,0.000000,0.000000,7887.000000,0.020503,0.220010,69.615963,1.000000
max,100.000000,100.000000,2819.000000,8326.000000,50.102000,66.720000,52.681000,63.503000,0.430000,0.0,681856.000000,48127.000000,2645.000000,124801.000000,1602.028292,57494.434224,1288.000000,1.000000


### social

In [44]:
describeCategory(catCoinDF, 'social', useColumns)

Number of tokens: 166


id,sentiment_votes_up_percentage,sentiment_votes_down_percentage,market_cap_rank,coingecko_rank,coingecko_score,developer_score,community_score,liquidity_score,public_interest_score,facebook_likes,twitter_followers,reddit_subscribers,reddit_accounts_active_48h,telegram_channel_user_count,mean,std,kurt,trust_score
count,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000,166.0,1.660000e+02,166.000000,166.000000,166.000000,1.660000e+02,1.660000e+02,166.000000,166.000000
mean,40.497229,13.117229,934.253012,3103.265060,12.838066,5.654741,11.575548,9.851380,0.015867,0.0,1.742644e+05,655.277108,3.301205,5375.138554,2.089683e+13,3.589353e+14,136.670857,0.174699
std,44.141656,25.898890,944.344285,2094.419292,9.560229,14.113416,9.953629,14.087166,0.051885,0.0,1.080527e+06,3401.946073,14.367670,11444.772368,2.692369e+14,4.624558e+15,228.788559,0.721672
min,0.000000,0.000000,0.000000,59.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000,0.000000,0.000000,-2.514584e-02,0.000000e+00,0.000000,-1.000000
25%,0.000000,0.000000,0.000000,1426.500000,5.106750,0.000000,6.741750,1.000000,0.000000,0.0,5.490000e+02,0.000000,0.000000,0.000000,5.092983e-03,1.158821e-01,13.070207,0.000000
50%,0.000000,0.000000,692.000000,2654.500000,12.090000,0.000000,8.784500,1.000000,0.001000,0.0,5.071500e+03,0.000000,0.000000,677.500000,1.303104e-02,1.658554e-01,42.476834,0.000000
75%,86.257500,17.562500,1727.750000,4349.750000,18.247000,0.000000,13.166750,17.288500,0.007500,0.0,2.275100e+04,0.000000,0.000000,4449.750000,6.958373e-02,7.440676e-01,139.713251,1.000000
max,100.000000,100.000000,2880.000000,8238.000000,49.933000,65.218000,43.877000,63.503000,0.430000,0.0,1.005986e+07,33809.000000,169.000000,70503.000000,3.468874e+15,5.958326e+16,1386.896486,1.000000


### defi

In [45]:
describeCategory(catCoinDF, 'defi', useColumns)

Number of tokens: 127


id,sentiment_votes_up_percentage,sentiment_votes_down_percentage,market_cap_rank,coingecko_rank,coingecko_score,developer_score,community_score,liquidity_score,public_interest_score,facebook_likes,twitter_followers,reddit_subscribers,reddit_accounts_active_48h,telegram_channel_user_count,mean,std,kurt,trust_score
count,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.0,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000
mean,61.689921,19.412441,866.094488,1819.708661,19.747638,9.863488,11.522165,21.722024,0.070016,0.0,48745.905512,979.212598,6.669291,11622.559055,0.042663,0.754095,47.512117,0.763780
std,36.980122,23.711687,812.875305,1505.648181,11.385860,20.470293,9.175562,19.005945,0.237269,0.0,108204.227639,5199.639097,31.368535,17559.374835,0.225724,4.008871,124.024058,0.511094
min,0.000000,0.000000,0.000000,33.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.038561,0.009533,0.346538,-1.000000
25%,50.000000,0.000000,194.000000,765.000000,12.638500,0.000000,8.106500,1.255000,0.003000,0.0,4948.500000,0.000000,0.000000,1529.000000,-0.000303,0.092511,5.451322,1.000000
50%,76.190000,13.640000,699.000000,1327.000000,18.871000,0.000000,9.222000,17.193000,0.016000,0.0,17027.000000,0.000000,0.000000,4778.000000,0.003490,0.137273,12.022957,1.000000
75%,89.615000,27.415000,1283.000000,2500.500000,23.918000,0.000000,10.537500,33.921500,0.053000,0.0,42439.000000,0.000000,0.000000,14907.000000,0.014084,0.200409,31.918782,1.000000
max,100.000000,100.000000,2906.000000,7706.000000,56.009000,73.946000,45.991000,70.119000,2.306000,0.0,944515.000000,48119.000000,230.000000,99593.000000,1.837488,39.099211,927.848010,1.000000


### blockchain

In [46]:
describeCategory(catCoinDF, 'blockchain', useColumns)

Number of tokens: 108


id,sentiment_votes_up_percentage,sentiment_votes_down_percentage,market_cap_rank,coingecko_rank,coingecko_score,developer_score,community_score,liquidity_score,public_interest_score,facebook_likes,twitter_followers,reddit_subscribers,reddit_accounts_active_48h,telegram_channel_user_count,mean,std,kurt,trust_score
count,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.0,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000
mean,51.312130,19.058241,628.898148,1597.037037,24.634370,23.891324,22.007250,23.410269,0.013463,0.0,40451.490741,4633.361111,23.518519,10635.027778,inf,3.711868,102.598595,0.703704
std,40.142695,25.432925,703.681626,1880.283678,14.957121,30.110642,13.495543,19.337944,0.059043,0.0,77394.812328,12749.251643,47.760595,20928.422918,NaN,27.754256,241.404343,0.630372
min,0.000000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.011955,0.000000,0.000000,-1.000000
25%,0.000000,0.000000,0.000000,262.500000,14.932250,0.000000,8.683250,7.625000,0.001000,0.0,4770.750000,0.000000,0.000000,1489.500000,0.002077,0.086492,7.451221,1.000000
50%,67.425000,11.480000,366.500000,1031.000000,21.042000,0.000000,24.588000,20.739000,0.003500,0.0,12394.000000,296.500000,6.000000,4235.000000,0.006378,0.121054,15.759496,1.000000
75%,85.800000,27.595000,1089.750000,2027.500000,34.440000,57.648750,33.235500,41.851750,0.009000,0.0,33794.250000,2858.500000,21.000000,9803.000000,0.015863,0.204610,52.775820,1.000000
max,100.000000,100.000000,2499.000000,8149.000000,62.936000,84.683000,47.453000,73.649000,0.606000,0.0,495631.000000,89213.000000,295.000000,155342.000000,inf,285.347638,1270.482933,1.000000


### business

In [49]:
describeCategory(catCoinDF, 'business', useColumns)

Number of tokens: 186


id,sentiment_votes_up_percentage,sentiment_votes_down_percentage,market_cap_rank,coingecko_rank,coingecko_score,developer_score,community_score,liquidity_score,public_interest_score,facebook_likes,twitter_followers,reddit_subscribers,reddit_accounts_active_48h,telegram_channel_user_count,mean,std,kurt,trust_score
count,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.0,186.000000,186.000000,186.000000,186.000000,1.860000e+02,1.860000e+02,186.000000,186.000000
mean,33.837957,14.011505,870.403226,2905.677419,14.586984,10.341785,14.440995,10.154070,0.003747,0.0,14709.940860,1051.623656,6.370968,5432.338710,5.908090e+28,7.089708e+29,187.363095,0.193548
std,43.166668,28.702127,933.049800,2252.427004,11.124302,17.915820,11.137464,14.119079,0.007165,0.0,24703.040831,4827.999767,18.570742,12573.839613,8.057561e+29,9.669073e+30,263.109580,0.828650
min,0.000000,0.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-2.078912e-02,2.074884e-02,0.274368,-1.000000
25%,0.000000,0.000000,0.000000,1172.500000,5.711250,0.000000,7.342500,1.000000,0.000000,0.0,1546.000000,0.000000,0.000000,187.250000,4.782525e-03,1.196583e-01,15.520548,-1.000000
50%,0.000000,0.000000,599.500000,2376.000000,13.113000,0.000000,8.883000,1.000000,0.001000,0.0,5680.000000,0.000000,0.000000,1965.500000,1.417327e-02,2.028535e-01,57.524682,0.000000
75%,79.642500,13.355000,1600.500000,4112.500000,19.931500,13.636500,24.407500,17.415750,0.003000,0.0,15505.250000,219.750000,6.000000,4287.500000,9.894632e-02,1.021757e+00,255.340724,1.000000
max,100.000000,100.000000,2934.000000,8281.000000,56.991000,89.368000,43.487000,60.550000,0.050000,0.0,158551.000000,58011.000000,169.000000,90058.000000,1.098905e+31,1.318686e+32,1175.524164,1.000000


### meme

In [47]:
describeCategory(catCoinDF, 'meme', useColumns)

Number of tokens: 51


id,sentiment_votes_up_percentage,sentiment_votes_down_percentage,market_cap_rank,coingecko_rank,coingecko_score,developer_score,community_score,liquidity_score,public_interest_score,facebook_likes,twitter_followers,reddit_subscribers,reddit_accounts_active_48h,telegram_channel_user_count,mean,std,kurt,trust_score
count,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.00000,51.000000,51.000000,51.0,5.100000e+01,5.100000e+01,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000
mean,74.276667,21.801765,342.392157,3495.745098,10.713020,0.365039,16.73098,16.353627,0.021608,0.0,8.408035e+04,4.620031e+04,136.313725,23053.196078,0.013336,0.254365,85.730319,0.235294
std,22.220538,16.847929,631.544807,1997.939317,9.018925,2.606901,15.21532,17.307375,0.046546,0.0,2.982493e+05,2.955914e+05,646.554514,44436.229522,0.038157,0.697619,201.559615,0.472789
min,0.000000,0.000000,0.000000,125.000000,0.200000,0.000000,0.00000,1.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,-0.016649,0.015225,0.289747,-1.000000
25%,65.425000,8.710000,0.000000,1553.000000,3.600500,0.000000,7.60000,1.000000,0.002000,0.0,2.169000e+03,0.000000e+00,0.000000,1496.500000,-0.000327,0.054093,11.370824,0.000000
50%,77.780000,21.000000,0.000000,3646.000000,7.621000,0.000000,9.24300,10.618000,0.004000,0.0,7.623000e+03,0.000000e+00,0.000000,5669.000000,0.001848,0.079133,22.259937,0.000000
75%,89.345000,33.330000,440.500000,4951.500000,17.502500,0.000000,24.99000,24.532000,0.015500,0.0,2.077600e+04,3.375000e+02,4.000000,18909.000000,0.004253,0.122357,53.744994,0.500000
max,100.000000,75.000000,2193.000000,7695.000000,43.920000,18.617000,71.22300,70.897000,0.211000,0.0,1.930095e+06,2.106870e+06,3827.000000,192985.000000,0.169055,4.582374,1346.030462,1.000000
